# PI VI — EPI (YOLOv8) • Roteiro Avançado (Google Colab)Pipeline completo: instalação, Drive, dataset, checagem, treino, retomada, validação, predições e export.

## 0) Instalação limpa (PyTorch + Ultralytics) e checagem de GPU

In [ ]:
!pip -q uninstall -y torch torchvision torchaudio ultralytics > /dev/null 2>&1
try:
    import nvidia; gpu = True
except Exception:
    gpu = False
if gpu:
    !pip -q install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121
else:
    !pip -q install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cpu
!pip -q install ultralytics opencv-python pandas tensorboard
import torch, ultralytics, torchvision
print("torch:", torch.__version__, "| cuda?", torch.cuda.is_available(), "| GPUs:", torch.cuda.device_count())
print("ultralytics:", ultralytics.__version__, "| torchvision:", torchvision.__version__)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.9/780.9 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 109.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 1) Montar Drive e definir pastas

In [ ]:
from google.colab import drive
from pathlib import Path
drive.mount('/content/drive', force_remount=True)
BASE = Path('/content/drive/MyDrive/PI_VI_EPI'); DATA = BASE/'data'; RUNS = BASE/'runs'
for p in [DATA, RUNS]: p.mkdir(parents=True, exist_ok=True)
print("BASE:", BASE); print("DATA:", DATA); print("RUNS:", RUNS)


Mounted at /content/drive
BASE: /content/drive/MyDrive/PI_VI_EPI
DATA: /content/drive/MyDrive/PI_VI_EPI/data
RUNS: /content/drive/MyDrive/PI_VI_EPI/runs


## 2) (Opcional) Upload do dataset .zip + normalização (YOLO)

In [ ]:
from google.colab import files; uploaded = files.upload()
import zipfile, glob, os, shutil
from pathlib import Path
zip_names = [k for k in uploaded.keys() if k.lower().endswith('.zip')]; assert zip_names, "Nenhum .zip enviado."
zip_name = zip_names[0]
TMP = Path('/content/_ds_tmp'); shutil.rmtree(TMP, ignore_errors=True); TMP.mkdir(exist_ok=True)
with zipfile.ZipFile(zip_name,'r') as zf: zf.extractall(TMP)
def find_root(root: Path):
    for p in [root]+[d for d in root.rglob('*') if d.is_dir()]:
        if (p/'images'/'train').exists() or (p/'train'/'images').exists() or (p/'valid'/'images').exists() or (p/'val'/'images').exists(): return p
    return root
SRC = find_root(TMP); print("SRC:", SRC)
IM, LB = (BASE/'data/images'), (BASE/'data/labels')
for sp in ['train','val','test']: (IM/sp).mkdir(parents=True, exist_ok=True); (LB/sp).mkdir(parents=True, exist_ok=True)
def copytree(src, dst):
    if src and src.exists():
        for rt,_,files in os.walk(src):
            for f in files:
                s = Path(rt)/f; d = dst/f; d.parent.mkdir(parents=True, exist_ok=True); shutil.copy2(s,d)
if (SRC/'images'/'train').exists():
    copytree(SRC/'images'/'train', IM/'train'); copytree(SRC/'labels'/'train', LB/'train')
    if (SRC/'images'/'val').exists():  copytree(SRC/'images'/'val', IM/'val'); copytree(SRC/'labels'/'val', LB/'val')
    if (SRC/'images'/'test').exists(): copytree(SRC/'images'/'test', IM/'test'); copytree(SRC/'labels'/'test', LB/'test')
elif (SRC/'train'/'images').exists():
    copytree(SRC/'train'/'images', IM/'train'); copytree(SRC/'train'/'labels', LB/'train')
    val_im = (SRC/'valid'/'images') if (SRC/'valid'/'images').exists() else (SRC/'val'/'images')
    val_lb = (SRC/'valid'/'labels') if (SRC/'valid'/'labels').exists() else (SRC/'val'/'labels')
    if val_im.exists(): copytree(val_im, IM/'val'); copytree(val_lb, LB/'val')
    if (SRC/'test'/'images').exists(): copytree(SRC/'test'/'images', IM/'test'); copytree(SRC/'test'/'labels', LB/'test')
print("Normalização concluída em:", BASE/'data')


KeyboardInterrupt: 

In [ ]:
import glob
from pathlib import Path

# Define o caminho para a pasta de dados
DATA = Path('/content/drive/MyDrive/PI_VI_EPI/data')

# Adicione esta linha para criar o diretório se ele não existir
DATA.mkdir(exist_ok=True, parents=True)

# Agora, o script pode criar o arquivo data.yaml dentro do diretório recém-criado
(DATA/'data.yaml').write_text(f"path: {DATA}\ntrain: images/train\nval: images/val\ntest: images/test\nnames: [hardhat, person]")

# Funções de contagem
def count_imgs(p):
    return sum(len(glob.glob(str(p/e))) for e in ('*.jpg','*.jpeg','*.png','*.bmp','*.webp'))

def count_txt(p):
    return len(glob.glob(str(p/'*.txt')))

## 3) data.yaml + checagem

In [ ]:
from pathlib import Path, PurePosixPath
import glob
DATA = Path('/content/drive/MyDrive/PI_VI_EPI/data')
(DATA/'data.yaml').write_text(f"path: {DATA}\ntrain: images/train\nval: images/val\ntest: images/test\nnames: [hardhat, person]")
def count_imgs(p): return sum(len(glob.glob(str(p/e))) for e in ('*.jpg','*.jpeg','*.png','*.bmp','*.webp'))
def count_txt(p): return len(glob.glob(str(p/'*.txt')))
IM_TRAIN, IM_VAL, IM_TEST = DATA/'images/train', DATA/'images/val', DATA/'images/test'
LB_TRAIN, LB_VAL, LB_TEST = DATA/'labels/train', DATA/'labels/val', DATA/'labels/test'
print((DATA/'data.yaml').read_text())
print("imgs/train:", count_imgs(IM_TRAIN), "| lbls/train:", count_txt(LB_TRAIN))
print("imgs/val  :", count_imgs(IM_VAL),   "| lbls/val  :", count_txt(LB_VAL))
print("imgs/test :", count_imgs(IM_TEST),  "| lbls/test :", count_txt(LB_TEST))


path: /content/drive/MyDrive/PI_VI_EPI/data
train: images/train
val: images/val
test: images/test
names: [hardhat, person]
imgs/train: 9177 | lbls/train: 9177
imgs/val  : 0 | lbls/val  : 0
imgs/test : 1766 | lbls/test : 1766


## 4) Treino avançado

In [ ]:
import torch
from ultralytics import YOLO
device = '0' if torch.cuda.is_available() else 'cpu'
DATA_YAML_PATH = '/content/drive/MyDrive/PI_VI_EPI/data/data.yaml'
from pathlib import Path
RUNS = Path('/content/drive/MyDrive/PI_VI_EPI/runs'); RUNS.mkdir(parents=True, exist_ok=True)
model = YOLO('yolov8s.pt')
results = model.train(
    data=DATA_YAML_PATH, epochs=100 if device!='cpu' else 30, imgsz=640 if device!='cpu' else 512, batch=16 if device!='cpu' else 8,
    device=device, project=str(RUNS), name='detect_epiv_adv', resume=False, save_period=5, patience=20, workers=2, seed=42,
    optimizer='auto', cos_lr=True, lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=5e-4, amp=True,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, fliplr=0.5, flipud=0.0, mosaic=1.0, close_mosaic=10, copy_paste=0.0, erasing=0.4, cache=False,
)


Ultralytics 8.3.195 🚀 Python-3.12.11 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/PI_VI_EPI/data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=detect_epiv_adv, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, plots=Tr

RuntimeError: Dataset '/content/drive/MyDrive/PI_VI_EPI/data/data.yaml' error ❌ Dataset '/content/drive/MyDrive/PI_VI_EPI/data/data.yaml' images not found, missing path '/content/drive/MyDrive/PI_VI_EPI/data/images/val'
Note dataset download directory is '/content/datasets'. You can update this in '/root/.config/Ultralytics/settings.json'

## 5) Retomar, validar, predições e export

In [ ]:
from ultralytics import YOLO
from pathlib import Path
from glob import glob
import torch

# Define o dispositivo (GPU ou CPU)
device = '0' if torch.cuda.is_available() else 'cpu'
print(f"Usando dispositivo: {device}")

# --- CORREÇÃO PRINCIPAL ---
# Apontando para a pasta do treinamento BEM-SUCEDIDO
RUNS = Path('/content/drive/MyDrive/PI_VI_EPI/runs')
RUN_NAME = 'detect_epiv_adv5' # <--- ESTA É A LINHA MAIS IMPORTANTE
# --- FIM DA CORREÇÃO ---

# Caminho completo para o melhor modelo
BEST_MODEL_PATH = RUNS / RUN_NAME / 'weights' / 'best.pt'

print(f"\n--- Carregando e Validando o modelo: {BEST_MODEL_PATH} ---")

# Carrega o modelo correto
model = YOLO(str(BEST_MODEL_PATH))

# Roda a validação
metrics = model.val(data='/content/drive/MyDrive/PI_VI_EPI/data/data.yaml', device=device)
print("\n--- Métricas de Validação do Modelo Correto ---")
print(metrics)


print("\n--- Executando predições em 10 imagens de amostra ---")
VAL_DIR = Path('/content/drive/MyDrive/PI_VI_EPI/data/images/val')
samples = glob(str(VAL_DIR / '*.jpg'))[:10]

# Roda a predição
model.predict(
    source=samples,
    device=device,
    imgsz=640,
    conf=0.5, # Aumentei a confiança para ter menos detecções de baixa qualidade
    save=True,
    project=str(RUNS),
    name=f'{RUN_NAME}_pred_val_final' # Novo nome para não sobrescrever
)

print(f"\nPredições salvas em: {RUNS}/{RUN_NAME}_pred_val_final")


Usando dispositivo: 0

--- Carregando e Validando o modelo: /content/drive/MyDrive/PI_VI_EPI/runs/detect_epiv_adv5/weights/best.pt ---
WARNING ⚠️ /content/drive/MyDrive/PI_VI_EPI/runs/detect_epiv_adv5/weights/best.pt appears to require 'torch.utils.serialization', which is not in Ultralytics requirements.
AutoInstall will run now for 'torch.utils.serialization' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official Ultralytics model, i.e. 'yolo predict model=yolo11n.pt'


ModuleNotFoundError: No module named 'torch.utils.serialization'

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/PI_VI_EPI/data/images/train

000001_jpg.rf.b323367c8ffffd332ac3768c4fb58cd7.jpg
000001_jpg.rf.d8dc4f100ddb3a76f2ad90ddbb638e21.jpg
000002_jpg.rf.24da71a86fca864b865b450ac73c7591.jpg
000002_jpg.rf.a77cde44f8bcc6475b7a0d0ab9c6bee8.jpg
000003_jpg.rf.91699e5926b6528b27c8304d0a6dd69a.jpg
000004_jpg.rf.05cf0defebca15297bdb7e038031fc84.jpg
000004_jpg.rf.9466bb37fd6a0ada1347623bf7a40aec.jpg
000005_jpg.rf.99790e4c08ee9763c7cd612e0758d56b.jpg
000005_jpg.rf.ff03e59cec01d34da23095e6c02f15d2.jpg
000006_jpg.rf.23e746459def6a20f0f1008803f41a01.jpg
000006_jpg.rf.df41ded7e24597625c0a0fdb951e992c.jpg
000007_jpg.rf.032d94da3c0ca4026ff324338b69b46a.jpg
000007_jpg.rf.d2b450b67ff9e9aaf84af2425a05e646.jpg
000008_jpg.rf.5dffaa8894f04ba7fa933fa32e4ebb41.jpg
000009_jpg.rf.791fe8961891afc85b99fd62615e74d7.jpg
000009_jpg.rf.8d11242b29b0a1784b5f91e23e12e061.jpg
000010_jpg.rf.4974508f48776ba2f85f903b206e6f78.jpg
000010_jpg.rf.70cc02b3b5d0f77583327ceb897b42c3.jpg
000011_jpg.rf.0d910c620ac03afbb55a21574b672197.jpg
000011_jpg.rf.9cfd485d68af0b1e1

In [ ]:
!ls /content/drive/MyDrive/PI_VI_EPI/data/images/val

In [ ]:
!ls /content/drive/MyDrive/PI_VI_EPI/data/images/test

005298_jpg.rf.ed576b4eb48fdfd5729d7f7f8b40dca2.jpg
005299_jpg.rf.f96a3eadb6b2bee472f173082af1a121.jpg
005300_jpg.rf.cf9bacc042236e6092c8dd251acd37b6.jpg
005301_jpg.rf.7161730c4524c05f203fd4e327cea037.jpg
005302_jpg.rf.4d4e02212d659967cc0b36b53a026e1a.jpg
005303_jpg.rf.46d09a22237c38fa49d577ae1a28c94b.jpg
005304_jpg.rf.bfdf06b216e1dbc6adb30a340d642896.jpg
005305_jpg.rf.53b145d469cdddadcd37ebcfa37cfd79.jpg
005306_jpg.rf.7a75e76829c7b95e70af3ba5cb70aec3.jpg
005307_jpg.rf.7ec568d753ee4361c9921b5720b2f75d.jpg
005308_jpg.rf.6aa94c29911a808b3b1a0dbe5bb778c8.jpg
005309_jpg.rf.fc94e37b6da1281496bbbbd3861925a3.jpg
005310_jpg.rf.408978aee7a6fc91b3539523182611e1.jpg
005311_jpg.rf.3908384b6ccdf9936958cb537f9685e1.jpg
005312_jpg.rf.a088d641d661caf0b6f86629ae84e8eb.jpg
005313_jpg.rf.babb42e32ef2dbe6664e86f967e10a4a.jpg
005314_jpg.rf.99c854023c70268f6a8ee5402d799f7f.jpg
005315_jpg.rf.2188ca1011a17c97277252ce7d105ea8.jpg
005316_jpg.rf.4561e64db6d5ed7aaeeb514d1dd51ba0.jpg
005317_jpg.rf.b89d2f742f591dd5a

In [ ]:
from pathlib import Path
DATA = Path('/content/drive/MyDrive/PI_VI_EPI/data')
(DATA/'data.yaml').write_text(f"path: {DATA}\ntrain: images/train\nval: images/test\ntest: images/test\nnames: [hardhat, person]")

123

In [ ]:
!yolo task=detect mode=train model=yolov8n.pt data=/content/drive/MyDrive/PI_VI_EPI/data/data.yaml epochs=100 imgsz=640 project=/content/drive/MyDrive/PI_VI_EPI/runs device=0

Ultralytics 8.3.199 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/PI_VI_EPI/data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train9, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, 